In [1]:
1+1

2

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

In [4]:
#도구정의하기
from langchain_community.tools import TavilySearchResults

query = "스테이크와 어울리는 와인을 추천해주세요."

#Tavily 검색 도구 이용
web_search = TavilySearchResults(max_results=2)

search_results = web_search.invoke(query)    #도구를 가지고 호출하도록 

for result in search_results:
    print(result)
    print('-'*20)
    

{'url': 'https://m.blog.naver.com/wineislikeacat/223096696241', 'content': '중간에서 높음 사이의 적당한 탄닌과 산도를 가진 와인을 선택하면\n\n씹는 맛의 긴 여운 속 느끼함을 잡아줄 수 있습니다.\n\n\u200b\n\n이에 맞는 레드와인으로는\n\n카베르네 소비뇽(Carbernet Sauvignon), 시라(Syrah) 품종을 추천드려요!\n\n\u200b\n\n![Image 7](https://mblogthumb-phinf.pstatic.net/MjAyMzA1MDhfMTEw/MDAxNjgzNTQ1MDQ3NDI0.v4ZHSvlETicMdag0DaoWGFsrTMFezoiZD_41J0Htwhsg.mPiLhr9soLZMXEOb_GJatpnqVUUNFZkrcmQRn9omaewg.JPEG.wineislikeacat/clark-douglas-7WJV-SDSj6c-unsplash.jpg?type=w80_blur)\n\n안심 스테이크와 어울리는 와인\n\n> **안심 스테이크와 어울리는 와인 품종은? 산지오베제!**\n\n고기 본연의 맛을 즐기기 가장 좋은 부위로 꼽히는 안심은 [...] ![Image 6](https://mblogthumb-phinf.pstatic.net/MjAyMzA1MDhfMTQ4/MDAxNjgzNTQ0MjY4OTc2.FasX2MpcBaCDnf0qjWA_PcbLTIlySdPLi7FFnJlTboog.AQwrp1etohVQ3fiIHA0ga9lD45uGexCOZzB_1C12eGQg.JPEG.wineislikeacat/paul-einerhand-dwtt3noJG2k-unsplash.jpg?type=w800)\n\n등심 & 채끝 스테이크와 어울리는 와인\n\n> **등심 스테이크, 채끝 스테이크와 어울리는 와인 품종은? 카베르네 소비뇽, 시라 등!**\n\n등심은 소의 등뼈를 둘러싸고 있는 부위입니다.\n\n육질이 연하고 지방이 많아서 두꺼운 스테이크 구이용으로 최적이죠.\n\n채끝은 등뼈에서 허리

In [5]:
print('자료형: ')
print(type(web_search))
print('='*10)

print('name: ')
print(web_search.name)
print('='*10)

print('description: ')
print(web_search.description)
print('='*10)

print('schema: ')
print(web_search.args_schema.schema())
print('='*10)


자료형: 
<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
name: 
tavily_search_results_json
description: 
A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.
schema: 
{'description': 'Input for the Tavily tool.', 'properties': {'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}, 'required': ['query'], 'title': 'TavilyInput', 'type': 'object'}


In [6]:
#tool calling

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

#binding
llm_with_tools = llm.bind_tools(tools=[web_search])
#외부지식을 검색하는건 같음. 문서(벡터화)가 아니라 툴을 전달하는 것이 다름. 

In [7]:
#도구 호출이 필요 없는 경우

query = "안녕하세요."
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print('='*10)

pprint(ai_msg.content)
print('='*10)

pprint(ai_msg.tool_calls)
print('='*10)


AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 82, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'stop', 'logprobs': None}, id='run--4edae248-1f7a-41d7-99ad-a4b490336916-0', usage_metadata={'input_tokens': 82, 'output_tokens': 11, 'total_tokens': 93})
'안녕하세요! 어떻게 도와드릴까요?'
[]


In [8]:
#도구 호출이 필요한 경우

query = "스테이크와 어울리는 와인을 추천해주세요."

#웹검색을 해야한다고 판단 ? or not?
ai_msg = llm_with_tools.invoke(query)

pprint(ai_msg)
print('='*10)

pprint(ai_msg.content)
print('='*10)

pprint(ai_msg.tool_calls)
print('='*10)


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_40K24Z1Y2xh4JSdMintQ5jPQ', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 91, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4c607939-056d-411d-90d4-cb92acfad790-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_40K24Z1Y2xh4JSdMintQ5jPQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 27, 'total_tokens': 118})
''
[{'args': {'query': '스테이크와 어울리는 와인 추천'},
  'id

In [9]:
tool_call = ai_msg.tool_calls[0]
tool_call

{'name': 'tavily_search_results_json',
 'args': {'query': '스테이크와 어울리는 와인 추천'},
 'id': 'call_40K24Z1Y2xh4JSdMintQ5jPQ',
 'type': 'tool_call'}

In [11]:
#도구 직접 호출

tool_message = web_search.invoke(tool_call)

print(tool_message)

content='[{"url": "https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", "content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 대표주자\\n------------------------------------------------\\n\\n![Image 3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image 4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\\n\\n**추천 이유:**\\n\\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, 고기의 감칠맛을 더욱 살려줍니다.\\n\\n**어울리는 스테이크:**\\n\\n*   리브아이 스테이크\\n*   뉴욕 스트립\\n*   티본 스테이크\\n\\n**추천 와인:**\\n\\n*   나파 밸리 카베르네 소비뇽\\n*   샤또 마고 (Château Margaux) [...] ![Image 10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image 11](https://blog.kakaocdn.net/dn/bX2dCB/btsL48cPEez/PDKprxC9mea7bnWY9Y65bk/img.webp)![Image 12

In [12]:
pprint(tool_message.content)

('[{"url": '
 '"https://alcohol.hobby-tech.com/entry/%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%99%80-%EC%B0%B0%EB%96%A1%EA%B6%81%ED%95%A9-%EC%8A%A4%ED%85%8C%EC%9D%B4%ED%81%AC%EC%97%90-%EC%96%B4%EC%9A%B8%EB%A6%AC%EB%8A%94-%EC%99%80%EC%9D%B8-5%EA%B0%80%EC%A7%80", '
 '"content": "1. 카베르네 소비뇽 (Cabernet Sauvignon) – 스테이크 와인의 '
 '대표주자\\n------------------------------------------------\\n\\n![Image '
 '3](https://blog.kakaocdn.net/dn/xWKr4/btsL30NJ5Zo/7JzT5nGQ6CXB4sTxlpdxf1/img.png)![Image '
 '4](https://blog.kakaocdn.net/dn/R0nXK/btsL4WXUW0Y/eQYYP6Pj1WKewJVFnM5ycK/img.png)\\n\\n**추천 '
 '이유:**\\n\\n카베르네 소비뇽은 강한 탄닌과 풍부한 바디감이 특징인 레드 와인입니다. 이는 스테이크의 기름진 육즙과 균형을 이루며, '
 '고기의 감칠맛을 더욱 살려줍니다.\\n\\n**어울리는 스테이크:**\\n\\n*   리브아이 스테이크\\n*   뉴욕 스트립\\n*   '
 '티본 스테이크\\n\\n**추천 와인:**\\n\\n*   나파 밸리 카베르네 소비뇽\\n*   샤또 마고 (Château '
 'Margaux) [...] ![Image '
 '10](https://blog.kakaocdn.net/dn/dnoiCg/btsL5OybTaE/91vDmwi70ApQ1FZK6DFRQ0/img.png)![Image '
 '11](https://blog.kakaocdn.net/dn/bX2dCB/btsL48cPEez/PDKpr

In [17]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

today = datetime.today().strftime("%Y-%m-%d")
today

prompt = ChatPromptTemplate.from_messages([
    ("system", f"You are helpful AI assistant. Today's date is {today}"),
    ("human", "{user_input}"),
    ("placeholder", "{message}"),
])

llm = ChatOpenAI(model="gpt-4o-mini")

llm_with_tools = llm.bind_tools(tools = [web_search])

llm_chain = prompt | llm_with_tools

@chain
def web_search_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    #도구사용할지말지 (도구에 대한 정보보)
    print("ai_msg: \n", ai_msg)
    print("=*10")
    
    tool_msgs = web_search.batch(ai_msg.tool_calls, config=config) 
    #rough한 데이터 툴로 검색한 결과값 툴(태블리)이 검색한 결과값만 
    print("tool_message: \n", tool_message)
    print("=*10")
    
    return llm_chain.invoke({**input_, "message": [ai_msg, *tool_msgs]}, config=config)
                    # 니가 부른 툴, 툴이 검색한 정보 전부 종합해서 니가 판단해봐. 최적의 답을 포맷해서 보여줘
                    
response = web_search_chain.invoke("오늘 모엣샹동 샴페인의 가격은 얼마인가요?") 
            #함수지만 Runnable 넣어서 invoke사용 가능함
            
pprint(response.content)

    


ai_msg: 
 content='' additional_kwargs={'tool_calls': [{'id': 'call_m6CTlXhI7lmFX8st5V3keeoh', 'function': {'arguments': '{"query":"모엣샹동 샴페인 가격 2025"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 113, 'total_tokens': 144, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--4faa666d-e31b-4a07-856c-603adf2cfccd-0' tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '모엣샹동 샴페인 가격 2025'}, 'id': 'call_m6CTlXhI7lmFX8st5V3keeoh', 'type': 'tool_call'}] usage_metadata={'input_tokens': 113, 'output_tokens': 31, 'total_tokens': 144}
=*10
tool_message: 
 content='[{"url": "https://alcoh

In [24]:
#도구 정의하기
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool
from typing import List

#툴 만들기
@tool
def search_web(query: str) -> str:      #쿼리가 들어오면 문자열로 답
    """
    Searches the internet for information 
    that does not exist in the database or 
    for the latest information.
    """
    
    tavily_search = TavilySearchResults(max_results=2)
    docs = tavily_search.invoke(query)
    
    # print(docs)
    formatted_docs = "\n--\n".join([
        f"<Document href='{doc['url']}'>\n{doc['content']}\n</Document>"
        for doc in docs
            
    ])
    
    if len(formatted_docs) > 0:
        return formatted_docs
    
    return "관련 정보를 찾을 수 없습니다."



In [35]:
query = "스테이크와 어울리는 와인을 추천해주세요"
search_result = search_web.invoke(query)
# pprint(formatted_docs)
print(search_result)

<Document href='https://m.blog.naver.com/wineislikeacat/223096696241'>
중간에서 높음 사이의 적당한 탄닌과 산도를 가진 와인을 선택하면

씹는 맛의 긴 여운 속 느끼함을 잡아줄 수 있습니다.

​

이에 맞는 레드와인으로는

카베르네 소비뇽(Carbernet Sauvignon), 시라(Syrah) 품종을 추천드려요!

​

![Image 7](https://mblogthumb-phinf.pstatic.net/MjAyMzA1MDhfMTEw/MDAxNjgzNTQ1MDQ3NDI0.v4ZHSvlETicMdag0DaoWGFsrTMFezoiZD_41J0Htwhsg.mPiLhr9soLZMXEOb_GJatpnqVUUNFZkrcmQRn9omaewg.JPEG.wineislikeacat/clark-douglas-7WJV-SDSj6c-unsplash.jpg?type=w80_blur)

안심 스테이크와 어울리는 와인

> **안심 스테이크와 어울리는 와인 품종은? 산지오베제!**

고기 본연의 맛을 즐기기 가장 좋은 부위로 꼽히는 안심은 [...] ![Image 6](https://mblogthumb-phinf.pstatic.net/MjAyMzA1MDhfMTQ4/MDAxNjgzNTQ0MjY4OTc2.FasX2MpcBaCDnf0qjWA_PcbLTIlySdPLi7FFnJlTboog.AQwrp1etohVQ3fiIHA0ga9lD45uGexCOZzB_1C12eGQg.JPEG.wineislikeacat/paul-einerhand-dwtt3noJG2k-unsplash.jpg?type=w800)

등심 & 채끝 스테이크와 어울리는 와인

> **등심 스테이크, 채끝 스테이크와 어울리는 와인 품종은? 카베르네 소비뇽, 시라 등!**

등심은 소의 등뼈를 둘러싸고 있는 부위입니다.

육질이 연하고 지방이 많아서 두꺼운 스테이크 구이용으로 최적이죠.

채끝은 등뼈에서 허리를 감싸는 뼈로 내려오는 쪽의 살인데요,

마블링 러버들의 선택을 쉽사리 받는

In [37]:
llm_with_tools = llm.bind_tools(tools=[search_web])

query = "스테이크와 어울리는 와인을 추천해주세요"
search_result = search_web.invoke(query)

pprint(ai_msg)
print('='*10)

pprint(ai_msg.content)
print('='*10)

pprint(ai_msg.tool_calls)
print('='*10)


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_40K24Z1Y2xh4JSdMintQ5jPQ', 'function': {'arguments': '{"query":"스테이크와 어울리는 와인 추천"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 91, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4c607939-056d-411d-90d4-cb92acfad790-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '스테이크와 어울리는 와인 추천'}, 'id': 'call_40K24Z1Y2xh4JSdMintQ5jPQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 27, 'total_tokens': 118})
''
[{'args': {'query': '스테이크와 어울리는 와인 추천'},
  'id

In [38]:
from langchain.document_loaders import TextLoader

# 메뉴판 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_menu.txt", encoding="utf-8")
documents = loader.load()

print(len(documents))

1


In [39]:
from langchain_core.documents import Document

# 문서 분할 (Chunking)
def split_menu_items(document):
    """
    메뉴 항목을 분리하는 함수 
    """
    # 정규표현식 정의 
    pattern = r'(\d+\.\s.*?)(?=\n\n\d+\.|$)'
    menu_items = re.findall(pattern, document.page_content, re.DOTALL)
    
    # 각 메뉴 항목을 Document 객체로 변환
    menu_documents = []
    for i, item in enumerate(menu_items, 1):
        # 메뉴 이름 추출
        menu_name = item.split('\n')[0].split('.', 1)[1].strip()
        
        # 새로운 Document 객체 생성
        menu_doc = Document(
            page_content=item.strip(),
            metadata={
                "source": document.metadata['source'],
                "menu_number": i,
                "menu_name": menu_name
            }
        )
        menu_documents.append(menu_doc)
    
    return menu_documents


# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 시그니처 스테이크
내용:
1. 시그니처 스테이크
   • 가격: ₩35,000
   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
   • 설명: 셰프의 특제 시그니처 메뉴로, ...

메뉴 번호: 2
메뉴 이름: 트러플 리조또
내용:
2. 트러플 리조또
   • 가격: ₩22,000
   • 주요 식재료: 이탈리아산 아르보리오 쌀, 블랙 트러플, 파르미지아노 레지아노 치즈
   • 설명: 크리미한 텍스처의 리조...


In [45]:
# Chroma Vectorstore를 사용하기 위한 준비
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
)

# Chroma 인덱스 생성
menu_db = Chroma.from_documents(
    documents=menu_documents,
    embedding=embeddings_model,
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

# Retriever 생성
menu_retriever = menu_db.as_retriever(
    search_kwargs={'k': 2},
)

# 쿼리 테스트
query = "시그니처 스테이크의 가격과 특징은 무엇인가요?"
docs = menu_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

검색 결과: 2개
메뉴 번호: 1
메뉴 이름: 시그니처 스테이크

메뉴 번호: 8
메뉴 이름: 안심 스테이크 샐러드



In [42]:
# 와인 메뉴 텍스트 데이터를 로드
loader = TextLoader("./data/restaurant_wine.txt", encoding="utf-8")
documents = loader.load()

# 메뉴 항목 분리 실행
menu_documents = []
for doc in documents:
    menu_documents += split_menu_items(doc)

# 결과 출력
print(f"총 {len(menu_documents)}개의 메뉴 항목이 처리되었습니다.")
for doc in menu_documents[:2]:
    print(f"\n메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print(f"내용:\n{doc.page_content[:100]}...")


# Chroma 인덱스 생성
wine_db = Chroma.from_documents(
    documents=menu_documents,
    embedding=embeddings_model,
    collection_name="restaurant_wine",
    persist_directory="./chroma_db",
)

wine_retriever = wine_db.as_retriever(
    search_kwargs={'k': 2},
)

query = "스테이크와 어울리는 와인을 추천해주세요."
docs = wine_retriever.invoke(query)
print(f"검색 결과: {len(docs)}개")

for doc in docs:
    print(f"메뉴 번호: {doc.metadata['menu_number']}")
    print(f"메뉴 이름: {doc.metadata['menu_name']}")
    print()

총 10개의 메뉴 항목이 처리되었습니다.

메뉴 번호: 1
메뉴 이름: 샤토 마고 2015
내용:
1. 샤토 마고 2015
   • 가격: ₩450,000
   • 주요 품종: 카베르네 소비뇽, 메를로, 카베르네 프랑, 쁘띠 베르도
   • 설명: 보르도 메독 지역의 프리미엄 ...

메뉴 번호: 2
메뉴 이름: 돔 페리뇽 2012
내용:
2. 돔 페리뇽 2012
   • 가격: ₩380,000
   • 주요 품종: 샤르도네, 피노 누아
   • 설명: 프랑스 샴페인의 대명사로 알려진 프레스티지 큐베입니다. 시트러스...
검색 결과: 2개
메뉴 번호: 10
메뉴 이름: 그랜지 2016

메뉴 번호: 9
메뉴 이름: 샤토 디켐 2015



In [43]:
# 벡터 저장소 로드
menu_db = Chroma(
    embedding_function=embeddings_model,
    collection_name="restaurant_menu",
    persist_directory="./chroma_db",
)

@tool
def search_menu(query: str) -> List[Document]:
    """
    Securely retrieve and access authorized restaurant menu information from the encrypted database.
    Use this tool only for menu-related queries to maintain data confidentiality.
    """
    docs = menu_db.similarity_search(query, k=2)
    if len(docs) > 0:
        return docs

    return [Document(page_content="관련 메뉴 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(search_menu))
print("-"*100)

print("name: ")
print(search_menu.name)
print("-"*100)

print("description: ")
pprint(search_menu.description)
print("-"*100)

print("schema: ")
pprint(search_menu.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_menu
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant menu information from the '
 'encrypted database.\n'
 'Use this tool only for menu-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant menu '
                'information from the encrypted database.\n'
                'Use this tool only for menu-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'search_menu',
 'type': 'object'}
----------------

In [47]:
from langchain_core.tools import tool
from typing import List
from langchain_core.documents import Document

# 벡터 저장소 로드
wine_db = Chroma(
   embedding_function=embeddings_model,
   collection_name="restaurant_wine",
   persist_directory="./chroma_db",
)

@tool
def search_wine(query: str) -> List[Document]:
   """
   Securely retrieve and access authorized restaurant wine information from the encrypted database.
   Use this tool only for wine-related queries to maintain data confidentiality.
   """
   docs = wine_db.similarity_search(query, k=2)
   if len(docs) > 0:
      return docs

   return [Document(page_content="관련 와인 정보를 찾을 수 없습니다.")]

# 도구 속성
print("자료형: ")
print(type(search_wine))
print("-"*100)

print("name: ")
print(search_wine.name)
print("-"*100)

print("description: ")
pprint(search_wine.description)
print("-"*100)

print("schema: ")
pprint(search_wine.args_schema.schema())
print("-"*100)

자료형: 
<class 'langchain_core.tools.structured.StructuredTool'>
----------------------------------------------------------------------------------------------------
name: 
search_wine
----------------------------------------------------------------------------------------------------
description: 
('Securely retrieve and access authorized restaurant wine information from the '
 'encrypted database.\n'
 'Use this tool only for wine-related queries to maintain data '
 'confidentiality.')
----------------------------------------------------------------------------------------------------
schema: 
{'description': 'Securely retrieve and access authorized restaurant wine '
                'information from the encrypted database.\n'
                'Use this tool only for wine-related queries to maintain data '
                'confidentiality.',
 'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'search_wine',
 'type': 'object'}
----------------

In [48]:
# LLM에 도구를 바인딩 (2개의 도구 바인딩)
llm_with_tools = llm.bind_tools(tools=[search_menu, search_wine])

# 도구 호출이 필요한 LLM 호출을 수행
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요."
ai_msg = llm_with_tools.invoke(query)

# LLM의 전체 출력 결과 출력
pprint(ai_msg)
print("-" * 100)

# 메시지 content 속성 (텍스트 출력)
pprint(ai_msg.content)
print("-" * 100)

# LLM이 호출한 도구 정보 출력
pprint(ai_msg.tool_calls)
print("-" * 100)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_iqWsPUgh7PefSjkhg8q01H6n', 'function': {'arguments': '{"query": "시그니처 스테이크"}', 'name': 'search_menu'}, 'type': 'function'}, {'id': 'call_qniypVZvL0H2CKYetgXst2tb', 'function': {'arguments': '{"query": "스테이크"}', 'name': 'search_wine'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 137, 'total_tokens': 190, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--db9b20ab-7ce4-4c5b-a366-2751d95459c5-0', tool_calls=[{'name': 'search_menu', 'args': {'query': '시그니처 스테이크'}, 'id': 'call_iqWsPUgh7PefSjkhg8q01H6n', 'type': 'tool_call'}, {'name': 'search_wine', 'args': {'query

In [49]:
tools = [search_web, search_wine, search_menu]
for tool in tools:
    print(tool.name)

search_web
search_wine
search_menu


In [ ]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 
prompt = ChatPromptTemplate([
    ("system", f"You are a helpful AI assistant. Today's date is {today}."),
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

# ChatOpenAI 모델 초기화 
llm = ChatOpenAI(model="gpt-4o-mini")

# 4개의 검색 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=tools)

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

# 도구 실행 체인 정의
@chain
def restaurant_menu_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:     #여러개의 질의문이기 때문
        print(f"{tool_call['name']}: \n{tool_call}")
        print("-"*100)

        if tool_call["name"] == "search_web":
            tool_message = search_web.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        # elif tool_call["name"] == "wiki_summary":
        #     tool_message = wiki_summary.invoke(tool_call, config=config)
        #     tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_wine":
            tool_message = search_wine.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)

        elif tool_call["name"] == "search_menu":
            tool_message = search_menu.invoke(tool_call, config=config)
            tool_msgs.append(tool_message)            

    #tool_message 만으로도 답변으로 사용해도 되지만 가공되지 않은 데이터임. 

    print("tool_msgs: \n", tool_msgs)
    print("-"*100)
    return llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

# 체인 실행
response = restaurant_menu_chain.invoke("시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인 추천도 해주세요.")

# 응답 출력 
print(response.content)

search_menu: 
{'name': 'search_menu', 'args': {'query': '시그니처 스테이크 가격과 특징'}, 'id': 'call_wzbe1cqZC56ORLAiMmxEOWEK', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
search_wine: 
{'name': 'search_wine', 'args': {'query': '스테이크에 어울리는 와인 추천'}, 'id': 'call_WyGVPudy4tismO2YgCHOoOQh', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\\n   • 가격: ₩35,000\\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '안심 스테이크 샐러드', 'menu_number': 8, 'source': './data/restaurant_menu.txt'}, page_content='8. 안심 스테이크 샐러드\\n 

In [54]:
response = restaurant_menu_chain.invoke("스테이크의 유래에 대해서 알려주세요")
print(response.content)

tool_msgs: 
 []
----------------------------------------------------------------------------------------------------
스테이크의 유래는 유럽, 특히 영국과 프랑스에 뿌리를 두고 있습니다. 스테이크는 일반적으로 소고기 또는 다른 고기의 잘라낸 부위를 의미하며, 고기를 구워서 조리한 요리입니다.

스테이크라는 용어는 고대 영어 "steik"에서 유래되었으며, 이는 "자르다" 또는 "썰다"라는 뜻을 가지고 있습니다. 16세기에는 스테이크라는 용어가 등장하였고, 당시에는 고기를 굽는 방식이 오늘날과는 다소 달랐습니다.

프랑스에서도 스테이크 개념이 발전하였으며, 다양한 조리법과 소스가 추가되어 스테이크 요리가 더욱 다양해졌습니다. 특히 프랑스의 "스테이크 프리트"와 같은 요리가 유명합니다.

19세기 중반에 스테이크는 미국으로 전파되어 큰 인기를 얻었으며, 이후 각 지역의 요리 스타일에 맞게 변형되었습니다. 현재는 세계 여러 나라에서 다양한 방식으로 조리되고 즐겨지고 있습니다.


In [55]:
response = restaurant_menu_chain.invoke("전세계에서 가장 비싼 스테이크에 대해서 알려주세요")
print(response.content)

search_web: 
{'name': 'search_web', 'args': {'query': '세계에서 가장 비싼 스테이크 2025'}, 'id': 'call_jQzsbigfKA1ZqSQOOGg8EYbP', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="<Document href='https://www.youtube.com/shorts/yEOo-7uuxbs'>\n[스미스앤웰렌스키] #조니워커 #조니워커페어링 #조니워커블루 #조니워커더블블랙 #맛집유튜버 #조니워커골드리저브 #조니워커그린라벨 #위스키\n</Document>\n--\n<Document href='https://blog.naver.com/pjh5101/223733990996'>\nWagyu Tomahawk 스테이크는 Rhode Island 레스토랑에서 제공되는 최고의 식사 메뉴이다. 가격은 109 달러. 이 레스토랑은 거의 2년 전에 이 요리를 제공\n</Document>", name='search_web', tool_call_id='call_jQzsbigfKA1ZqSQOOGg8EYbP')]
----------------------------------------------------------------------------------------------------



In [52]:
# 체인 실행
response = restaurant_menu_chain.invoke("파스타 메뉴가 있나요? 이 음식의 역사 또는 유래를 알려주세요.")

# 응답 출력 
print(response.content)

search_menu: 
{'name': 'search_menu', 'args': {'query': '파스타'}, 'id': 'call_lRdB7Sl1GsmjjykuojQejm7q', 'type': 'tool_call'}
----------------------------------------------------------------------------------------------------
tool_msgs: 
 [ToolMessage(content="[Document(metadata={'menu_name': '해산물 파스타', 'menu_number': 6, 'source': './data/restaurant_menu.txt'}, page_content='6. 해산물 파스타\\n   • 가격: ₩24,000\\n   • 주요 식재료: 링귀네 파스타, 새우, 홍합, 오징어, 토마토 소스\\n   • 설명: 알 덴테로 삶은 링귀네 파스타에 신선한 해산물을 듬뿍 올린 메뉴입니다. 토마토 소스의 산미와 해산물의 감칠맛이 조화를 이루며, 마늘과 올리브 오일로 풍미를 더했습니다. 파슬리를 뿌려 향긋한 맛을 더합니다.'), Document(metadata={'menu_name': '샤토 마고 2015', 'menu_number': 1, 'source': './data/restaurant_wine.txt'}, page_content='1. 샤토 마고 2015\\n   • 가격: ₩450,000\\n   • 주요 품종: 카베르네 소비뇽, 메를로, 카베르네 프랑, 쁘띠 베르도\\n   • 설명: 보르도 메독 지역의 프리미엄 와인으로, 깊고 복잡한 풍미가 특징입니다. 블랙커런트, 블랙베리의 과실향과 함께 시더, 담배, 가죽 노트가 어우러집니다. 탄닌이 부드럽고 균형 잡힌 구조를 가지며, 긴 여운이 인상적입니다. 숙성 잠재력이 뛰어나 10-20년 이상 보관이 가능합니다.')]", name='search_menu', tool_call_id='call_lRdB7Sl1Gsmj

In [50]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate

examples = [
    HumanMessage("트러플 리조또의 가격과 특징, 그리고 어울리는 와인에 대해 알려주세요.", name="example_user"),
    AIMessage("메뉴 정보를 검색하고, 위키피디아에서 추가 정보를 찾은 후, 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_menu", "args": {"query": "트러플 리조또"}, "id": "1"}]),
    ToolMessage("트러플 리조또: 가격 ₩28,000, 이탈리아 카나롤리 쌀 사용, 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리", tool_call_id="1"),
    AIMessage("트러플 리조또의 가격은 ₩28,000이며, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 이제 추가 정보를 위키피디아에서 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "wiki_summary", "args": {"query": "트러플 리조또", "k": 1}, "id": "2"}]),
    ToolMessage("트러플 리조또는 이탈리아 요리의 대표적인 리조또 요리 중 하나로, 고급 식재료인 트러플을 사용하여 만든 크리미한 쌀 요리입니다. 주로 아르보리오나 카나롤리 등의 쌀을 사용하며, 트러플 오일이나 생 트러플을 넣어 조리합니다. 리조또 특유의 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 것이 특징입니다.", tool_call_id="2"),
    AIMessage("트러플 리조또의 특징에 대해 알아보았습니다. 이제 어울리는 와인을 검색해보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[{"name": "search_wine", "args": {"query": "트러플 리조또에 어울리는 와인"}, "id": "3"}]),
    ToolMessage("트러플 리조또와 잘 어울리는 와인으로는 주로 중간 바디의 화이트 와인이 추천됩니다. 1. 샤르도네: 버터와 오크향이 트러플의 풍미를 보완합니다. 2. 피노 그리지오: 산뜻한 산미가 리조또의 크리미함과 균형을 이룹니다. 3. 베르나차: 이탈리아 토스카나 지방의 화이트 와인으로, 미네랄리티가 트러플과 잘 어울립니다.", tool_call_id="3"),
    AIMessage("트러플 리조또(₩28,000)는 이탈리아의 대표적인 리조또 요리 중 하나로, 이탈리아 카나롤리 쌀을 사용하고 블랙 트러플 향과 파르메산 치즈를 듬뿍 넣어 조리합니다. 주요 특징으로는 크리미한 질감과 트러플의 강렬하고 독특한 향이 조화를 이루는 점입니다. 고급 식재료인 트러플을 사용해 풍부한 맛과 향을 내며, 주로 아르보리오나 카나롤리 등의 쌀을 사용합니다. 트러플 리조또와 잘 어울리는 와인으로는 중간 바디의 화이트 와인이 추천됩니다. 특히 버터와 오크향이 트러플의 풍미를 보완하는 샤르도네, 산뜻한 산미로 리조또의 크리미함과 균형을 이루는 피노 그리지오, 그리고 미네랄리티가 트러플과 잘 어울리는 이탈리아 토스카나 지방의 베르나차 등이 좋은 선택이 될 수 있습니다.", name="example_assistant"),
]

system = """You are an AI assistant providing restaurant menu information and general food-related knowledge.
For information about the restaurant's menu, use the search_menu tool.
For other general information, use the wiki_summary tool.
For wine recommendations or pairing information, use the search_wine tool.
If additional web searches are needed or for the most up-to-date information, use the search_web tool.
"""

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{query}"),
])

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini")

# 검색 도구를 직접 LLM에 바인딩 가능
llm_with_tools = llm.bind_tools(tools=tools)

# Few-shot 프롬프트를 사용한 체인 구성
fewshot_search_chain = few_shot_prompt | llm_with_tools

# 체인 실행
query = "스테이크 메뉴가 있나요? 스테이크와 어울리는 와인을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'search_menu', 'args': {'query': '스테이크'}, 'id': 'call_pTcWuNgTOBMVDo3tiHTMyKRE', 'type': 'tool_call'}
{'name': 'search_wine', 'args': {'query': '스테이크에 어울리는 와인'}, 'id': 'call_2k838AotF5YfAA5RXZFim3cY', 'type': 'tool_call'}


In [57]:
# 체인 실행
query = "파스타의 유래에 대해서 알고 있나요? 서울 강남의 파스타 맛집을 추천해주세요."
response = fewshot_search_chain.invoke(query)

# 결과 출력
for tool_call in response.tool_calls:
    print(tool_call)

{'name': 'search_web', 'args': {'query': '파스타의 유래'}, 'id': 'call_nWpP3bt89aju7BUcaubdYRlX', 'type': 'tool_call'}
{'name': 'search_menu', 'args': {'query': '서울 강남 파스타 맛집'}, 'id': 'call_RTihHZpg4VCqgmAahM88jMs8', 'type': 'tool_call'}


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

agent_prompt = ChatPromptTemplate.from_messages([
    ("system", dedent("""
        You are an AI assistant providing restaurant menu information and general food-related knowledge. 
        Your main goal is to provide accurate information and effective recommendations to users.

        Key guidelines:
        1. For restaurant menu information, use the search_menu tool. This tool provides details on menu items, including prices, ingredients, and cooking methods.
        2. For general food information, history, and cultural background, utilize the wiki_summary tool.
        3. For wine recommendations or food and wine pairing information, use the search_wine tool.
        4. If additional web searches are needed or for the most up-to-date information, use the search_web tool.
        5. Provide clear and concise responses based on the search results.
        6. If a question is ambiguous or lacks necessary information, politely ask for clarification.
        7. Always maintain a helpful and professional tone.
        8. When providing menu information, describe in the order of price, main ingredients, and distinctive cooking methods.
        9. When making recommendations, briefly explain the reasons.
        10. Maintain a conversational, chatbot-like style in your final responses. Be friendly, engaging, and natural in your communication.


        Remember, understand the purpose of each tool accurately and use them in appropriate situations. 
        Combine the tools to provide the most comprehensive and accurate answers to user queries. 
        Always strive to provide the most current and accurate information.
        """)),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),      #순회 계속해서 툴 호출 더 할지 최종결정할지 
])

In [62]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

tools = [search_web, search_menu, search_wine]
agent = create_tool_calling_agent(llm, tools, agent_prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [64]:
query = "시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인도 추천해주세요."

agent_response = agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `search_menu` with `{'query': '시그니처 스테이크'}`


[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '안심 스테이크 샐러드', 'menu_number': 8, 'source': './data/restaurant_menu.txt'}, page_content='8. 안심 스테이크 샐러드\n   • 가격: ₩26,000\n   • 주요 식재료: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈\n   • 설명: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.')]
Invoking: `search_wine` with `{'query': '스테이크'}`


[Document(metadata={'menu_name': '샤토 디켐 2015', 'menu_number': 9, 'source': './data/restaurant_wine.txt'}, page_content='9. 샤토 디켐 2015\n   • 가격: ₩800,000 (375ml)\n   • 주요 품종: 세미

In [65]:
pprint(agent_response)

{'input': '시그니처 스테이크의 가격과 특징은 무엇인가요? 그리고 스테이크와 어울리는 와인도 추천해주세요.',
 'output': '### 시그니처 스테이크 정보\n'
           '- **가격**: ₩35,000\n'
           '- **주요 재료**: 소고기, 로즈마리, 적양파\n'
           '- **특징**: 이 스테이크는 정통 시그니처 요리로, 21일 간 숙성된 소고기를 사용하여 부드럽고 풍부한 맛을 '
           '자랑합니다. 적양파와 로즈마리로 조리하여 향미가 더해지며, 고기가 육즙이 풍부하게 유지됩니다.\n'
           '\n'
           '### 스테이크와 어울리는 와인 추천\n'
           '1. **엘리트 피트 2015**\n'
           '   - **가격**: ₩800,000 (375ml)\n'
           '   - **특징**: 이 와인은 뛰어난 품질의 레드 와인으로, 스테이크의 풍부한 맛과 잘 어울립니다. 과일 향과 함께 '
           '미세한 탄닌이 조화를 이루어, 고기와 함께 먹을 때 더욱 맛있습니다.\n'
           '\n'
           '2. **데이비드 빈 2019**\n'
           '   - **가격**: ₩95,000\n'
           '   - **특징**: 이 와인은 미디엄 바디의 레드 와인으로, 스테이크와 궁합이 좋습니다. 부드러운 과일향과 함께 '
           '뒷맛이 깔끔하여 고기의 기름진 맛을 중화시켜줍니다.\n'
           '\n'
           '스테이크와 함께 이 와인들을 곁들이면, 훌륭한 식사 경험을 누릴 수 있을 것입니다! 🍷🥩'}


In [ ]:
import gradio as gr
from typing import List, Tuple

def answer_invoke(message: str, history: List[Tuple[str, str]]) -> str:     #대화(질의) 할때마다 호출되는 함수 
    try:
        chat_history = []
        for human, ai in history:
            chat_history.append(HumanMessage(content=human))
            chat_history.append(AIMessage(content=ai))
            
        response = agent_executor.invoke({
            "input": message,        #이것만 넣어도 대답은 나오지만 맥락이 저장이 안됨. 
            "chat_history": chat_history[-2:],       #agent_prompt안에 있는 chat_history임 # 직전 두개(내질의, ai대답)만 넣겠다
            
        })
    
        return response['output']
    
    except Exception as e:
        print(f'Error 발생: {str(e)}')
        return "Sorry"
    

example_questions = [
    "시그니처 스테이크의 가격과 특징을 알려주세요.",
    "트러플 리조또와 잘 어울리는 와인을 추천해주세요.",
    "해산물 파스타의 주요 재료는 무엇인가요? 서울 강남 지역에 레스토랑을 추천해주세요.",
    "채식주의자를 위한 메뉴 옵션이 있나요?"
]

demo = gr.ChatInterface(
    fn=answer_invoke,
    title="레스토랑 메뉴 AI 어시스턴트",
    description="메뉴 정보 추천 등등",
    examples=example_questions,
    theme=gr.themes.Soft()
)

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.




> Entering new AgentExecutor chain...

Invoking: `search_menu` with `{'query': '시그니처 스테이크'}`


[Document(metadata={'menu_name': '시그니처 스테이크', 'menu_number': 1, 'source': './data/restaurant_menu.txt'}, page_content='1. 시그니처 스테이크\n   • 가격: ₩35,000\n   • 주요 식재료: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스\n   • 설명: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.'), Document(metadata={'menu_name': '안심 스테이크 샐러드', 'menu_number': 8, 'source': './data/restaurant_menu.txt'}, page_content='8. 안심 스테이크 샐러드\n   • 가격: ₩26,000\n   • 주요 식재료: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈\n   • 설명: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.')]시그니처 스테이크에 대한 정보는 다음과 같습니다:

- **가격**: ₩35,000
- **주요 식재료**: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스
- **특징**: 이 스테이크는 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와

In [ ]:
demo.close()